In [3]:
#***************** SCRIPT GLOBAL ***********************#
#libreias
import pandas as pd

## NODOS 
# PROVINCIAS
P = ["AZUAY", "LOJA", "IMBABURA", "BOLIVAR", "CAÑAR", "CARCHI", "CHIMBORAZO", "COTOPAXI", "EL ORO", "ESMERALDAS", "GALAPAGOS", "GUAYAS", "LOS RIOS", "MANABI", "MORONA SANTIAGO", "NAPO", "ORELLANA", "PASTAZA", "PICHINCHA", "SANTA ELENA", "SANTO DOMINGO DE LOS TSACHILAS","SUCUMBIOS", "TUNGURAHUA", "ZAMORA CHINCHIPE"]
# PROVINCIAS Y CANTONES DICCIONARIO DE LISTA DE CANTONES PARA CADA PROVINICA 

### Creacion de Parametros para los modelos de cada provincia.
## Tasa de Población
import pandas as pd
from gurobipy import * # Libreria de Solver Gurobi 
import numpy as np # Libreria para Matrices
import warnings

## Tasa de Poblacion
T1 = 8/100000 
T2 = 316 
## Modelo para cada provincia lee cada excel 

for p in P:
    ## Poblacion 
    Cantones = list(pd.read_excel(p +".xlsx", sheet_name='Frecuencia')["Nom_Can"])
    Poblacion = dict()
    i = 0 # Contador cantones
    for c in Cantones: 
        Poblacion[c] = list(pd.read_excel( p +".xlsx", sheet_name='Frecuencia')["Poblacion"])[i]
        i=i+1
    ## Demanda
    df = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})
    Fiscalias = list(df["Name"]) # Nombre de Fiscalías
    Grupos = ["Grupo1","Grupo2", "Grupo3" ]
    Demanda = tupledict({})
    Nombres_Y = tuplelist([])
    Nombres_X = tuplelist([])
    for c in Cantones:
        df1 = pd.read_excel( p +".xlsx", sheet_name=c)
        for g in Grupos: 
            j = 0 # Contador Fiscalías
            for f in Fiscalias:
                Demanda[c,g,Fiscalias[j]] = list(df1[g])[j]
                Nombres_Y.append((c,Fiscalias[j]))
                Nombres_X.append((c,g,Fiscalias[j]))
                j=j+1

    Nombres_X = list(set(Nombres_X))
    Nombres_Y =list(set(Nombres_Y))
    ## Carga Laboral
    Carga = tupledict({})
    df1 = pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')

    i = 0      # Contador de Cantones
    for c in Cantones:
        for f in Fiscalias:
            if f == 'FISCALIA DE SOLUCIONES RAPIDAS':
                Carga[c,f] = round(list(df1["TASA_SR"])[i])
                i=i+1
            else:
                Carga[c,f] = 316 

    ## Multicompetente 
    Multi = dict() 
    i = 0    # Contador de Cantones
    for c in Cantones:
        Multi[c] = list(df1["MULTICOMPETENTE"])[i]
        i=i+1

    de =[] # Demandas pronosticadas total 
    C1 = []
    C2 = []
    for c in Cantones:
        C2.append(c)
        t = pd.read_excel( p +".xlsx", sheet_name=c)
        de.append(t["Grupo1"].sum()+ t["Grupo2"].sum() + t["Grupo3"].sum())
        if t["Grupo1"].sum()+ t["Grupo2"].sum() + t["Grupo3"].sum() <= 158:
            C1.append(c)
            
    for c in C1:
        C2.remove(c)
    print(C2)
    ## *********************** CREACION DEL MODELO ***********************************

    ## Creamos el CONSTRUCTOR DEL MODELO 
    m = Model('Modelo_FGE')

    ## Creamos las Variables
    X = m.addVars(Nombres_X , name="", vtype=GRB.INTEGER) 
    Y = m.addVars(Nombres_Y , name="Fiscales_Necesarios", vtype=GRB.INTEGER)

    # Creacion de la FUNCIÓN OBJETIVO
    m.setObjective((Y.sum(Cantones,Fiscalias)) , GRB.MINIMIZE)

    ## Creacion de las Restricciones 
    # Restricción 1
    for c in C2:
        if Multi[c] == 0:
            for f in Fiscalias:
                m.addConstr((X.sum(c,Grupos,f) <= Y[c,f] * Carga[c,f]) , "tasa de carga")
        elif Multi[c] == 1:
            m.addConstr((X.sum(c, Grupos,Fiscalias) <= Y.sum(c,Fiscalias)*T2 ) , "tasa de carga_{}".format(f))

    # Restriccion 2      
    for f in Fiscalias:
        for c in C2:
            for g in Grupos:
                m.addConstr((X[c,g,f] == Demanda[c,g,f]), "demandadelitos")

    # Restriccion 3
    #for c in C2:
     #   m.addConstr(Y.sum(c,Fiscalias) >= Poblacion[c]* T1, "Se cumpla tasa T1")
    
    suma = 0
    for c in C1:
        suma = suma + Poblacion[c]
    
    m.addConstr(Y.sum(C1,Fiscalias) >= suma * T1, "Se cumpla tasa T1")
    m.addConstr((X.sum(C1, Grupos,Fiscalias) <= Y.sum(C1,Fiscalias)*T2 ), "Se cumpla tasa T1")
    
    m.optimize()

    warnings.simplefilter("ignore")
    ## Tabla de Resultados 1 
    Tabla1 = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})
    for c in Cantones:
        Res = []
        for f in Fiscalias:
            Res.append(Y[c,f].x) 
        Tabla1[c] = Res

    for c in Cantones:
        if Multi[c] == 0:
            Tabla1[c][1] = Tabla1[c][1]+Tabla1[c][8]
            Tabla1[c][8] = 0
        elif Multi[c] == 1:
            suma = 0 
            for j in range(len(list(Tabla1[c]))):
                suma = suma + Tabla1[c][j]
                Tabla1[c][j] = 0
            Tabla1[c][8] = suma

    ## Tabla de Resultados 2 
    Tabla2 = pd.DataFrame({'Cantones': Cantones})
    Fo = []
    for c in Cantones:
        Fo.append(Tabla1[c].sum())

    Tabla2["Poblacion"] = list(pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')["Poblacion"])
    Tabla2["Delitos"] = de
    Tabla2["Fiscales Actuales"] = list(pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')["Fisc_Actuales"])
    Tabla2["Fiscales x Carga"] = Fo
    Tabla2["Déficit x Carga"] = Tabla2["Fiscales x Carga"]-Tabla2["Fiscales Actuales"] 
    Tabla2["Tasa Actual"] = (Tabla2["Fiscales Actuales"] * 100000)/Tabla2["Poblacion"]
    Tabla2["Tasa x Carga"] = (Tabla2["Fiscales x Carga"] * 100000)/Tabla2["Poblacion"]

    
######################################## MODEL 2

    m = Model('Modelo_FGE')

    ## Creamos las Variables
    X = m.addVars(Nombres_X , name="", vtype=GRB.INTEGER) 
    Y = m.addVars(Nombres_Y , name="Fiscales_Necesarios", vtype=GRB.INTEGER)

    # Creacion de la FUNCIÓN OBJETIVO
    m.setObjective((Y.sum(Cantones,Fiscalias)) , GRB.MINIMIZE)

    ## Creacion de las Restricciones 
    # Restricción 1
    for c in C2:
        if Multi[c] == 0:
            for f in Fiscalias:
                m.addConstr((X.sum(c,Grupos,f) <= Y[c,f] * Carga[c,f]) , "tasa de carga")
        elif Multi[c] == 1:
            m.addConstr((X.sum(c, Grupos,Fiscalias) <= Y.sum(c,Fiscalias)*T2 ) , "tasa de carga_{}".format(f))

    # Restriccion 2      
    for f in Fiscalias:
        for c in C2:
            for g in Grupos:
                m.addConstr((X[c,g,f] == Demanda[c,g,f]), "demandadelitos")

    # Restriccion 3
    for c in C2:
        m.addConstr(Y.sum(c,Fiscalias) >= Poblacion[c]* T1, "Se cumpla tasa T1")
    
    suma = 0
    for c in C1:
        suma = suma + Poblacion[c]
    
    m.addConstr(Y.sum(C1,Fiscalias) >= suma * T1, "Se cumpla tasa T1")
    m.addConstr((X.sum(C1, Grupos,Fiscalias) <= Y.sum(C1,Fiscalias)*T2 ), "Se cumpla tasa T1")
    
    m.optimize()
    Tabla1 = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})

    warnings.simplefilter("ignore")
    ## Tabla de Resultados 1 
    for c in Cantones:
        Res = []
        for f in Fiscalias:
            Res.append(Y[c,f].x) 
        Tabla1[c] = Res

    for c in Cantones:
        if Multi[c] == 0:
            Tabla1[c][1] = Tabla1[c][1]+Tabla1[c][8]
            Tabla1[c][8] = 0
        elif Multi[c] == 1:
            suma = 0 
            for j in range(len(list(Tabla1[c]))):
                suma = suma + Tabla1[c][j]
                Tabla1[c][j] = 0
            Tabla1[c][8] = suma

    print(Tabla1)

    ## Tabla de Resultados 2 
    Fo = []
    for c in Cantones:
        Fo.append(Tabla1[c].sum())

    Tabla2["Fiscales x Población"] = Fo
    Tabla2["Déficit x Población"] = Tabla2["Fiscales x Población"]-Tabla2["Fiscales Actuales"] 
    Tabla2["Tasa x Población"] = (Tabla2["Fiscales x Población"] * 100000)/Tabla2["Poblacion"]
    print(Tabla2)
    
    dfNew = list(Tabla2["Déficit x Carga"])
    dfNew1 = list(Tabla2["Déficit x Población"] - Tabla2["Déficit x Carga"])
    k=0
    for i in dfNew1:
        if i < 0:
            dfNew1[k] = 0  
        k=k+1    
        
    Tabla1.loc[9] = ["Déficit x Carga"] + dfNew    
    Tabla1.loc[10] = ["Faltantes x Población"] + dfNew1 
    
    writer = pd.ExcelWriter('Resultados_' + p + '.xlsx')
    Tabla1.to_excel(writer, sheet_name='Fiscalias', index=False)
    Tabla2.to_excel(writer, sheet_name='Resumen', index=False)
    writer.save()
    writer.close()

['CUENCA', 'GUALACEO', 'CAMILO PONCE ENRIQUEZ', 'SANTA ISABEL', 'PAUTE', 'SIGSIG', 'GIRON', 'NABON', 'PUCARA', 'CHORDELEG']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 290 rows, 540 columns and 855 nonzeros
Model fingerprint: 0xf7b97a82
Variable types: 0 continuous, 540 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 59.0000000
Presolve removed 290 rows and 540 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 59 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.900000000000e+01, best bound 5.900000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.

Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 2e+03]
Found heuristic solution: objective 73.0000000
Presolve removed 337 rows and 576 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 73 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.300000000000e+01, best bound 7.300000000000e+01, gap 0.0000%
                      Name  LOJA  CATAMAYO  CALVAS  MACARA  SARAGURO  PALTAS  \
0     PERSONAS Y GARANTIAS   3.0       0.0     0.0     0.0       1.0     0.0   
1  DELINCUENCIA ORGANIZADA   3.0       0.0     0.0     0.0       1.0     0.0   
2   ADMINISTRACION PUBLICA   2.0       0.0     0.0     0.0       1.0     0.0   
3       SOLUCIONES RAPIDAS  16.0       0.0     0.0     0.0       1.0     0.0   
4      VIOLENCIA DE GENERO   6.0       0.

['GUARANDA', 'SAN MIGUEL', 'ECHANDíA', 'CALUMA', 'CHILLANES', 'SAN JOSE DE CHIMBO', 'LAS NAVES']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 206 rows, 252 columns and 441 nonzeros
Model fingerprint: 0xb29d79e4
Variable types: 0 continuous, 252 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective 23.0000000
Presolve removed 206 rows and 252 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 23 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.300000000000e+01, best bound 2.300000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 

['TULCAN', 'MONTUFAR', 'BOLIVAR', 'ESPEJO', 'MIRA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 150 rows, 216 columns and 360 nonzeros
Model fingerprint: 0xf59d864a
Variable types: 0 continuous, 216 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 6e+02]
Found heuristic solution: objective 20.0000000
Presolve removed 150 rows and 216 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using

['LATACUNGA', 'SAN MIGUEL DE SALCEDO', 'LA MANA', 'PUJILI', 'SAQUISILI', 'PANGUA', 'SIGCHOS']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 214 rows, 252 columns and 441 nonzeros
Model fingerprint: 0xe61e12e8
Variable types: 0 continuous, 252 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 36.0000000
Presolve removed 214 rows and 252 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 36 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 p

...

['MORONA', 'SUCUA', 'GUALAQUIZA', 'PALORA ( METZERA )', 'SANTIAGO (MENDEZ)', 'LIMON INDANZA', 'TAISHA', 'HUAMBOYA', 'LOGRONO', 'TIWINTZA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 290 rows, 432 columns and 720 nonzeros
Model fingerprint: 0xb8c4e2ca
Variable types: 0 continuous, 432 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 8e+02]
Found heuristic solution: objective 28.0000000
Presolve removed 290 rows and 432 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 28 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.800000000000e+01, best bound 2.800000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.

['PUERTO FRANCISCO DE ORELLANA', 'LA JOYA DE LOS SACHAS', 'LORETO']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 94 rows, 144 columns and 234 nonzeros
Model fingerprint: 0x6b135d4e
Variable types: 0 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 7e+02]
Found heuristic solution: objective 20.0000000
Presolve removed 94 rows and 144 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical pro

Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 266 rows, 288 columns and 576 nonzeros
Model fingerprint: 0x3bed040e
Variable types: 0 continuous, 288 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 282.0000000
Presolve removed 266 rows and 288 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 282 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.820000000000e+02, best bound 2.820000000000e+02, gap 0.0000%
                      Name  QUITO  RUMINAHUI  MEJIA  CAYAMBE  PEDRO MONCAYO  \
0     PERSONAS Y GARANTIAS    8.0        1.0    1.0      1.0            0.0   
1  DELINCUENCIA ORGANIZADA   12.0        1.0    1.0      1.0 

Model fingerprint: 0x81e4a2e4
Variable types: 0 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+03]
Found heuristic solution: objective 43.0000000
Presolve removed 39 rows and 72 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 43 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.300000000000e+01, best bound 4.300000000000e+01, gap 0.0000%
                      Name  SANTO DOMINGO  LA CONCORDIA
0     PERSONAS Y GARANTIAS            6.0           0.0
1  DELINCUENCIA ORGANIZADA            2.0           0.0
2   ADMINISTRACION PUBLICA            1.0           0.0
3       SOLUCIONES RAPIDAS           16.0           0.0
4      VIOLENCIA DE GENERO            5.0           0.0
5     PATRIMONIO CIUDADAN

  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 2e+03]
Found heuristic solution: objective 53.0000000
Presolve removed 213 rows and 324 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 53 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.300000000000e+01, best bound 5.300000000000e+01, gap 0.0000%
                      Name  AMBATO  SAN PEDRO DE PELILEO  BANOS DE AGUA SANTA  \
0     PERSONAS Y GARANTIAS     1.0                   0.0                  0.0   
1  DELINCUENCIA ORGANIZADA     2.0                   0.0                  0.0   
2   ADMINISTRACION PUBLICA     2.0                   0.0                  0.0   
3       SOLUCIONES RAPIDAS    14.0                   0.0                  0.0   
4      VIOLENCIA DE GENERO     5.0                   0.0                  0.0   
5     PATRIMO

In [30]:
a = pd.read_excel( "AZUAY"+".xlsx", sheet_name="CUENCA")
print(a)
print(a.iloc[0,2]+a.iloc[0,3]+a.iloc[0,4])

   Unnamed: 0   Fiscalias  Grupo1  Grupo2  Grupo3
0           1  Fiscalia 1     519       0       0
1           2  Fiscalia 2     401       0       0
2           3  Fiscalia 3      99      86     185
3           4  Fiscalia 4     589    2924    1733
4           5  Fiscalia 5    1273     117     855
5           6  Fiscalia 6     542      66     208
6           7  Fiscalia 7      37     247      15
7           8  Fiscalia 8     969     113     365
8           9  Fiscalia 9       0       0       0
519
